### Limpieza e inspección de los datos

Este notebook está esturcturado como sigue:

- **Importación de librerias**
- **Bloque A**: carga de los datos.
- **Bloque B**: inspección de los datos.
- **Bloque C**: limpieza y adecuación.
- **Bloque D**: exportación del conjunto resultante.

### Importación librerias

In [2]:
import pandas as pd
import re

import cld3

### BLOQUE A: Carga de los datos

**Pandas**: librería más popular de python que proporciona las herramientas y estructuras necesarias para manipular y analizar datos. La estructura básica de Pandas es el **DataFrame**, una colección ordenada de columnas con nombres y tipos, donde una sola fila representa un único caso (observación) y las columnas representan atributos particulares.

Guía: https://pandas.pydata.org/docs/getting_started/index.html

In [3]:
# Leemos el CSV que contiene los datos y lo cargamos en memoria
df = pd.read_csv("../data/amazon_reviews.csv")

In [6]:
df.shape

(12000, 3)

### BLOQUE B: inspección de los datos

El objetivo de la inspección es la familiarización con el conjunto de datos. Algunas preguntas iniciales que podría estar bien hacerse pueden ser:

- ¿En qué tipo de objeto están almacenados los datos?
- ¿Cuál es su dimensión?
- ¿Hay datos ausentes?
- ¿Que tipos diferentes de datos hay?
- ...

In [9]:
# Ver el tipo de objeto con el que estamos trabajando
type(df)

pandas.core.frame.DataFrame

In [10]:
# Dimensiones del dataframe
df.shape

(12000, 3)

In [11]:
# Otra manera más interesante de proporcionar la misma información anterior
print(f"Hay un dataframe de {df.shape[0]} filas y {df.shape[1]} columnas.")

Hay un dataframe de 12000 filas y 3 columnas.


In [12]:
# Resumen conciso del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 10646 to 16734
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   class_index   12000 non-null  int64 
 1   review_title  12000 non-null  object
 2   review_text   12000 non-null  object
dtypes: int64(1), object(2)
memory usage: 375.0+ KB


In [13]:
# Mostrar las primeras 5 filas
df.head()

,class_index,review_title,review_text
10646,1,Volume problems,I just got this player and it's been nothing b...
14819,1,Too Many Errors,It does seem to have all the food known to Cze...
292,1,too bulky,"They are SOOOO bulky, and it seems they don't ..."
23657,1,Bra,Very poor support and material is not of a goo...
19254,2,Coloring as Meditation...,"Scherek's ""Celestial Beings"" are simply heaven..."


In [14]:
# Mostrar las últimas 5 filas
df.tail()

,class_index,review_title,review_text
37,1,"OK, until it cracked",I thought I'd gotten a great deal on these att...
10954,2,Levis at a good price fast shipping,"The Levis were at a good price, the seller shi..."
17080,2,Love it,This is so helpful for my classroom. I am a to...
4372,1,Adidas axion shoes,"I bought these for aerobics class, but they di..."
16734,2,a pretty good buy,"The games in this book require few props, whic..."


In [15]:
# ¿Cuáles son los posibles valores que puede tomar la variable 'class_index'?
df['class_index'].unique()

array([1, 2])

La variable "class_index" indica el sentimiento de cada reseña:

- 1: sentimiento negativo
- 2: sentimiento positivo

### BLOQUE C: limpieza y adecuación

In [16]:
# Eliminación de las columnas que no son utiles a nuestro estudio
# Nos quedaremos solo con 'Revies'
df = df.drop(columns=['review_title'])
df.head()

,class_index,review_text
10646,1,I just got this player and it's been nothing b...
14819,1,It does seem to have all the food known to Cze...
292,1,"They are SOOOO bulky, and it seems they don't ..."
23657,1,Very poor support and material is not of a goo...
19254,2,"Scherek's ""Celestial Beings"" are simply heaven..."


In [17]:
""" 
Renombrar las columnas:
    - review_text -> text
    - class_index --> sentiment
"""
df.rename(columns={'review_text': 'text', 'class_index':'sentiment'}, inplace=True)

df.head()

,sentiment,text
10646,1,I just got this player and it's been nothing b...
14819,1,It does seem to have all the food known to Cze...
292,1,"They are SOOOO bulky, and it seems they don't ..."
23657,1,Very poor support and material is not of a goo...
19254,2,"Scherek's ""Celestial Beings"" are simply heaven..."


In [18]:
# Calcular la longitud de cada reseña
df['length'] = df['text'].apply(len)

In [19]:
# Resumen de la nueva columna "length"
df['length'].describe()

count    12000.000000
mean       401.550333
std        233.669965
min         31.000000
25%        205.000000
50%        352.000000
75%        559.000000
max       1006.000000
Name: length, dtype: float64

In [20]:
# busqueda de los elementos con longitud mínima (15)
df[df.length == 15]

,sentiment,text,length


In [21]:
# Vamos a remover aquellos textos que tiene más de 565 caracteres (Q3)
df = df[df.length <= 565]

In [22]:
# Comprobamos las dimensiones actuales del dataframe
df.shape

(9080, 3)

In [23]:
# Detección del idioma en el que está escrita cada reseña
df['language'] = df['text'].apply(lambda x: cld3.get_language(x)[0])

In [24]:
# ¿En que idioma están escritas las reseñas? ¿Cuantas reseñas tenemos para cada idioma?
df['language'].value_counts()

en    9050
es      23
fr       2
ha       1
pl       1
de       1
pt       1
so       1
Name: language, dtype: int64

In [25]:
# ¿Cual son las reseñas en español?
df[df.language=='es']

,sentiment,text,length,language
21274,2,He comprado varios de estos dicos para mi y pa...,144,es
13057,2,"Todo ha sido excelente, la entrega fue en el l...",142,es
21415,2,"Excelente producto, las peliculas son de buena...",150,es
21685,2,"Se refiere, básicamente, a cómo afirmar y mejo...",91,es
8130,1,"Me engañaron al comprar esto, los botones son ...",202,es
5246,2,¿Que tan util es la informacion en este libro?...,439,es
17244,1,"Es bastante incomodo, el boton hay q mantenerl...",241,es
23677,1,"Existen mejores sonidos, sonidos arm'nicos que...",443,es
7129,2,Al leer este libro en una clase de Maestría en...,247,es
5502,2,"Simplemente, sin igual. Me tomo 3 vueltas del ...",271,es


In [26]:
# Nos quedamos solo con aquellas que están escritas únicamente en inglés
df = df[df['language']=='en']

In [27]:
# Comprobamos las dimensiones actuales del dataframe
df.shape

(9050, 4)

### BLOQUE E: Exportación del conjunto resultante

En esta última fase exportamos y guardamos el conjunto de datos ya limpiado, procesado y adecuado en la carpeta 'data'. Solo guardaremos aquellas columnas que posteriormente nos serán útiles.
Ya que hay bastante datos nos vamo

In [28]:
# Eliminamos las columnas que ya no serán necesarias para los siguiente pasos
df = df.drop(columns=['language'])

In [29]:
# muestreo proporcional
df_sample = df.groupby('sentiment', group_keys=False).apply(
    lambda x: x.sample(frac=0.4, random_state=2023)
)

df_sample.head()

,sentiment,text,length
10083,1,I love some of Lora Leigh books so when I hear...,445
3140,1,Something is wrong with this DVD. It doesn't h...,473
17604,1,"The book contains good information, but it was...",330
17855,1,AND THAT'S EXACTLY WHAT RUTH PRICE IS!THAT LIT...,498
20661,1,Used this pressure cooker and thought it was w...,330


In [30]:
# vamos a combrobar que las proporcionese se hay mantenido iguales
print("Proporciones antes del muestreo:")
print(df['sentiment'].value_counts(normalize=True), "\n")

print("Proporciones despues del muestreo:")
print(df_sample['sentiment'].value_counts(normalize=True))

Proporciones antes del muestreo:
2    0.512265
1    0.487735
Name: sentiment, dtype: float64 

Proporciones despues del muestreo:
2    0.512155
1    0.487845
Name: sentiment, dtype: float64


In [31]:
# Guardamos los datos ya procesados en un fichero csv
df_sample.to_csv('../data/amazon_reviews_clean.csv', index=False)